In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 토큰을 직접 명시적으로 전달
hf_token = ""

c:\GitHubRepo\Medical-Artificial-Intelligence-Platform\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install huggingface_hub

In [3]:
!pip install -q langchain langchain-community sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

In [ ]:
# medical_vector_db.py
import os
import json
import logging
from datetime import datetime
from pathlib import Path
import concurrent.futures
import torch
from tqdm import tqdm

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class MedicalVectorStore:
    """
    의료 데이터를 위한 벡터 스토어 구축 클래스
    """
    def __init__(self, data_path="./medical_data", vector_store_path="./vector_stores", model_cache_dir="D:/huggingface_cache", use_gpu=True):
        """
        초기화 함수
        
        Args:
            data_path: 의료 데이터가 저장된 경로
            vector_store_path: 벡터 스토어를 저장할 경로
            model_cache_dir: 모델을 다운로드할 경로 (D드라이브)
            use_gpu: GPU 사용 여부
        """
        self.data_path = Path(data_path)
        self.vector_store_path = Path(vector_store_path)
        self.model_cache_dir = Path(model_cache_dir)

        # 기본 디렉토리 생성
        self.data_path.mkdir(parents=True, exist_ok=True)
        self.vector_store_path.mkdir(parents=True, exist_ok=True)
        self.model_cache_dir.mkdir(parents=True, exist_ok=True)

        # GPU 사용 여부 확인
        self.device = "cuda" if use_gpu and torch.cuda.is_available() else "cpu"
        logger.info(f"Using device: {self.device}")

        from langchain_community.embeddings import HuggingFaceEmbeddings

        # 한국어에 최적화된 가벼운 임베딩 모델 사용
        self.embeddings = HuggingFaceEmbeddings(
            model_name="meta-llama/Llama-4-Maverick-17B-128E-Instruct",  # 한국어에 최적화된 가벼운 모델
            model_kwargs={"device": self.device},
            cache_folder=str(self.model_cache_dir)  # 모델을 D드라이브에 저장
        )

        from langchain.text_splitter import RecursiveCharacterTextSplitter

        # 문서 분할기 설정
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            separators=["\n\n", "\n", ". ", " ", ""],
            length_function=len
        )

    def load_medical_data(self, file_pattern="*_patients.json"):
        """
        의료 데이터 로드 - 병렬 처리
        """
        import glob

        data_files = list(self.data_path.glob(file_pattern))

        if not data_files:
            logger.warning(f"No files matching {file_pattern} found in {self.data_path}")
            return []

        # 병렬 처리로 여러 파일 동시 로드
        documents = []

        with concurrent.futures.ThreadPoolExecutor(max_workers=min(len(data_files), os.cpu_count())) as executor:
            future_to_file = {
                executor.submit(self._load_single_file, file_path): file_path
                for file_path in data_files
            }

            for future in tqdm(concurrent.futures.as_completed(future_to_file),
                               total=len(future_to_file),
                               desc="Loading files"):
                file_docs = future.result()
                if file_docs:
                    documents.extend(file_docs)

        logger.info(f"Loaded {len(documents)} total documents from medical data")
        return documents

    def _load_single_file(self, file_path):
        """
        단일 파일 로드 (병렬 처리용)
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                patients = json.load(f)

            department = file_path.stem.replace("_patients", "")
            logger.info(f"Loading {len(patients)} patients from {department} department")

            # 각 환자 정보를 문서로 변환
            docs = []
            for patient in patients:
                docs.extend(self._convert_patient_to_documents(patient, department))

            return docs

        except Exception as e:
            logger.error(f"Error loading {file_path}: {e}")
            return []

    def _convert_patient_to_documents(self, patient, department):
        """
        환자 정보를 여러 개의 문서로 변환 (세분화된 정보)
        """
        from langchain.schema import Document

        documents = []

        # 환자 기본 정보 문서
        basic_info = f"""
        환자 ID: {patient['id']}
        이름: {patient['name']}
        성별: {patient['gender']}
        나이: {patient['age']}
        생년월일: {patient['birthdate']}
        혈액형: {patient.get('blood_type', '정보 없음')}
        키: {patient.get('height', '정보 없음')} cm
        체중: {patient.get('weight', '정보 없음')} kg
        BMI: {patient.get('bmi', '정보 없음')}
        주소: {patient.get('address', '정보 없음')}
        전화번호: {patient.get('phone', '정보 없음')}
        보험: {patient.get('insurance', '정보 없음')}
        진료과: {department}
        """

        if patient.get('allergies'):
            basic_info += f"\n알레르기: {', '.join(patient['allergies'])}"

        if patient.get('smoking'):
            basic_info += f"\n흡연: {patient['smoking']['status']}"
            if patient['smoking'].get('details'):
                basic_info += f" ({patient['smoking']['details']})"

        if patient.get('alcohol'):
            basic_info += f"\n음주: {patient['alcohol']['status']}"
            if patient['alcohol'].get('details'):
                basic_info += f" ({patient['alcohol']['details']})"

        documents.append(Document(
            page_content=basic_info.strip(),
            metadata={
                "patient_id": patient['id'],
                "name": patient['name'],
                "gender": patient['gender'],
                "age": patient['age'],
                "department": department,
                "document_type": "basic_info"
            }
        ))

        # 진단 정보 문서
        if patient.get('diagnoses'):
            for i, diagnosis in enumerate(patient['diagnoses']):
                diagnosis_doc = f"""
                환자 ID: {patient['id']}
                이름: {patient['name']}
                성별: {patient['gender']}
                나이: {patient['age']}

                [진단 정보 {i+1}]
                진단명: {diagnosis['name']}
                ICD10 코드: {diagnosis.get('icd10', '정보 없음')}
                진단일: {diagnosis.get('date', '정보 없음')}
                진단 의사: {diagnosis.get('doctor', '정보 없음')} (ID: {diagnosis.get('doctor_id', '정보 없음')})
                확신도: {diagnosis.get('confidence', '정보 없음')}
                상태: {diagnosis.get('status', '정보 없음')}
                중증도: {diagnosis.get('severity', '정보 없음')}
                메모: {diagnosis.get('memo', '정보 없음')}
                증상: {', '.join(diagnosis.get('symptoms', ['정보 없음']))}
                """

                documents.append(Document(
                    page_content=diagnosis_doc.strip(),
                    metadata={
                        "patient_id": patient['id'],
                        "name": patient['name'],
                        "gender": patient['gender'],
                        "age": patient['age'],
                        "department": department,
                        "document_type": "diagnosis",
                        "diagnosis_name": diagnosis['name'],
                        "diagnosis_date": diagnosis.get('date', ''),
                        "diagnosis_status": diagnosis.get('status', '')
                    }
                ))

        # 약물 정보 문서
        if patient.get('medications'):
            for i, medication in enumerate(patient['medications']):
                medication_doc = f"""
                환자 ID: {patient['id']}
                이름: {patient['name']}
                성별: {patient['gender']}
                나이: {patient['age']}

                [약물 정보 {i+1}]
                약물명: {medication['medication']}
                약물 분류: {medication.get('class', '정보 없음')}
                처방일: {medication.get('prescription_date', '정보 없음')}
                처방 기간: {medication.get('duration_days', '정보 없음')}일
                용량: {medication.get('dosage', '정보 없음')}
                빈도: {medication.get('frequency', '정보 없음')}
                재처방 횟수: {medication.get('refill', '정보 없음')}
                처방 의사: {medication.get('doctor', '정보 없음')} (ID: {medication.get('doctor_id', '정보 없음')})
                관련 진단: {medication.get('related_diagnosis', '정보 없음')}
                특별 지시사항: {medication.get('special_instructions', '정보 없음')}
                """

                documents.append(Document(
                    page_content=medication_doc.strip(),
                    metadata={
                        "patient_id": patient['id'],
                        "name": patient['name'],
                        "gender": patient['gender'],
                        "age": patient['age'],
                        "department": department,
                        "document_type": "medication",
                        "medication_name": medication['medication'],
                        "medication_class": medication.get('class', ''),
                        "related_diagnosis": medication.get('related_diagnosis', '')
                    }
                ))

        # 검사 결과 문서
        if patient.get('lab_results'):
            for i, lab in enumerate(patient['lab_results']):
                lab_doc = f"""
                환자 ID: {patient['id']}
                이름: {patient['name']}
                성별: {patient['gender']}
                나이: {patient['age']}

                [검사 결과 {i+1}]
                검사일: {lab.get('date', '정보 없음')}
                검사 유형: {lab.get('test_type', '정보 없음')}
                검사 요청 의사: {lab.get('ordering_doctor', '정보 없음')} (ID: {lab.get('ordering_doctor_id', '정보 없음')})
                검사 ID: {lab.get('lab_id', '정보 없음')}
                검체 채취 시간: {lab.get('collection_time', '정보 없음')}
                보고 시간: {lab.get('report_time', '정보 없음')}

                결과 항목:
                """

                for test_name, test_result in lab.get('results', {}).items():
                    lab_doc += f"""
                    - {test_name}: {test_result.get('value', '정보 없음')} {test_result.get('unit', '')}
                      (정상 범위: {test_result.get('normal_range', '정보 없음')})
                      {test_result.get('flag', '')}
                    """

                if lab.get('interpretation'):
                    lab_doc += f"\n해석: {lab['interpretation']}"

                documents.append(Document(
                    page_content=lab_doc.strip(),
                    metadata={
                        "patient_id": patient['id'],
                        "name": patient['name'],
                        "gender": patient['gender'],
                        "age": patient['age'],
                        "department": department,
                        "document_type": "lab_result",
                        "lab_date": lab.get('date', ''),
                        "test_type": lab.get('test_type', '')
                    }
                ))

        # 영상 검사 문서
        if patient.get('imaging_studies'):
            for i, study in enumerate(patient['imaging_studies']):
                imaging_doc = f"""
                환자 ID: {patient['id']}
                이름: {patient['name']}
                성별: {patient['gender']}
                나이: {patient['age']}

                [영상 검사 {i+1}]
                검사일: {study.get('date', '정보 없음')}
                검사 유형: {study.get('study_type', '정보 없음')}
                검사 요청 의사: {study.get('ordering_doctor', '정보 없음')} (ID: {study.get('ordering_doctor_id', '정보 없음')})
                영상의학과 의사: {study.get('radiologist', '정보 없음')}
                검사 ID: {study.get('study_id', '정보 없음')}

                소견: {study.get('findings', '정보 없음')}
                판독: {study.get('impression', '정보 없음')}
                추천: {study.get('recommendation', '정보 없음')}
                """

                documents.append(Document(
                    page_content=imaging_doc.strip(),
                    metadata={
                        "patient_id": patient['id'],
                        "name": patient['name'],
                        "gender": patient['gender'],
                        "age": patient['age'],
                        "department": department,
                        "document_type": "imaging_study",
                        "study_date": study.get('date', ''),
                        "study_type": study.get('study_type', '')
                    }
                ))

        # 시술 및 수술 문서
        if patient.get('procedures'):
            for i, procedure in enumerate(patient['procedures']):
                procedure_doc = f"""
                환자 ID: {patient['id']}
                이름: {patient['name']}
                성별: {patient['gender']}
                나이: {patient['age']}

                [시술/수술 {i+1}]
                시술일: {procedure.get('date', '정보 없음')}
                시술명: {procedure.get('name', '정보 없음')}
                설명: {procedure.get('description', '정보 없음')}
                시술 의사: {procedure.get('performing_doctor', '정보 없음')} (ID: {procedure.get('performing_doctor_id', '정보 없음')})
                시술 ID: {procedure.get('procedure_id', '정보 없음')}
                위치: {procedure.get('location', '정보 없음')}
                마취: {procedure.get('anesthesia', '정보 없음')}
                소요 시간: {procedure.get('duration_minutes', '정보 없음')}분
                결과: {procedure.get('outcome', '정보 없음')}
                """

                if procedure.get('complications'):
                    procedure_doc += f"\n합병증: {', '.join(procedure['complications'])}"

                procedure_doc += f"\n추적 관찰: {procedure.get('follow_up', '정보 없음')}"

                documents.append(Document(
                    page_content=procedure_doc.strip(),
                    metadata={
                        "patient_id": patient['id'],
                        "name": patient['name'],
                        "gender": patient['gender'],
                        "age": patient['age'],
                        "department": department,
                        "document_type": "procedure",
                        "procedure_date": procedure.get('date', ''),
                        "procedure_name": procedure.get('name', ''),
                        "outcome": procedure.get('outcome', '')
                    }
                ))

        # 진료 기록 문서
        if patient.get('visits'):
            for i, visit in enumerate(patient['visits']):
                visit_doc = f"""
                환자 ID: {patient['id']}
                이름: {patient['name']}
                성별: {patient['gender']}
                나이: {patient['age']}

                [진료 기록 {i+1}]
                방문 ID: {visit.get('visit_id', '정보 없음')}
                방문일: {visit.get('date', '정보 없음')}
                방문 시간: {visit.get('time', '정보 없음')}
                방문 유형: {visit.get('type', '정보 없음')}
                진료과: {visit.get('department', '정보 없음')}
                담당 의사: {visit.get('doctor', '정보 없음')} (ID: {visit.get('doctor_id', '정보 없음')})
                주 호소: {visit.get('chief_complaint', '정보 없음')}

                활력 징후:
                수축기 혈압: {visit.get('vital_signs', {}).get('systolic_bp', '정보 없음')} mmHg
                이완기 혈압: {visit.get('vital_signs', {}).get('diastolic_bp', '정보 없음')} mmHg
                맥박: {visit.get('vital_signs', {}).get('pulse', '정보 없음')} bpm
                체온: {visit.get('vital_signs', {}).get('temperature', '정보 없음')} °C
                호흡수: {visit.get('vital_signs', {}).get('respiratory_rate', '정보 없음')} /분
                산소포화도: {visit.get('vital_signs', {}).get('oxygen_saturation', '정보 없음')} %
                """

                if 'blood_glucose' in visit.get('vital_signs', {}):
                    visit_doc += f"혈당: {visit['vital_signs']['blood_glucose']} mg/dL\n"

                visit_doc += f"""
                임상 노트:
                주관적(S): {visit.get('clinical_note', {}).get('subjective', '정보 없음')}
                객관적(O): {visit.get('clinical_note', {}).get('objective', '정보 없음')}
                평가(A): {visit.get('clinical_note', {}).get('assessment', '정보 없음')}
                계획(P): {visit.get('clinical_note', {}).get('plan', '정보 없음')}

                진료 시간: {visit.get('duration_minutes', '정보 없음')}분
                """

                if visit.get('next_appointment'):
                    visit_doc += f"다음 예약: {visit['next_appointment']}"

                documents.append(Document(
                    page_content=visit_doc.strip(),
                    metadata={
                        "patient_id": patient['id'],
                        "name": patient['name'],
                        "gender": patient['gender'],
                        "age": patient['age'],
                        "department": department,
                        "document_type": "visit",
                        "visit_date": visit.get('date', ''),
                        "visit_type": visit.get('type', ''),
                        "chief_complaint": visit.get('chief_complaint', '')
                    }
                ))

        # 통합 문서 (전체 환자 기록을 하나의 문서로)
        integrated_doc = f"""
        [환자 통합 기록]
        환자 ID: {patient['id']}
        이름: {patient['name']}
        성별: {patient['gender']}
        나이: {patient['age']}
        생년월일: {patient['birthdate']}
        진료과: {department}

        [진단 요약]
        """

        if patient.get('diagnoses'):
            for diagnosis in patient['diagnoses']:
                integrated_doc += f"""
                - {diagnosis['name']} ({diagnosis.get('date', '날짜 없음')})
                  상태: {diagnosis.get('status', '정보 없음')}, 중증도: {diagnosis.get('severity', '정보 없음')}
                """
        else:
            integrated_doc += "진단 정보 없음\n"

        integrated_doc += "\n[약물 요약]\n"
        if patient.get('medications'):
            for med in patient['medications']:
                integrated_doc += f"""
                - {med['medication']} {med.get('dosage', '')} {med.get('frequency', '')}
                  처방일: {med.get('prescription_date', '정보 없음')}, 관련 진단: {med.get('related_diagnosis', '정보 없음')}
                """
        else:
            integrated_doc += "약물 정보 없음\n"

        integrated_doc += "\n[최근 검사 결과 요약]\n"
        if patient.get('lab_results'):
            # 가장 최근 검사 결과만 포함
            recent_lab = max(patient['lab_results'], key=lambda x: x.get('date', ''))
            integrated_doc += f"검사일: {recent_lab.get('date', '정보 없음')}, 검사 유형: {recent_lab.get('test_type', '정보 없음')}\n"

            for test_name, test_result in recent_lab.get('results', {}).items():
                flag = test_result.get('flag', '')
                if flag:
                    integrated_doc += f"- {test_name}: {test_result.get('value', '')} {test_result.get('unit', '')} ({flag})\n"
        else:
            integrated_doc += "검사 결과 정보 없음\n"

        integrated_doc += "\n[최근 방문 요약]\n"
        if patient.get('visits'):
            # 가장 최근 방문만 포함
            recent_visit = max(patient['visits'], key=lambda x: x.get('date', ''))
            integrated_doc += f"""
            방문일: {recent_visit.get('date', '정보 없음')}
            주 호소: {recent_visit.get('chief_complaint', '정보 없음')}
            평가: {recent_visit.get('clinical_note', {}).get('assessment', '정보 없음')}
            계획: {recent_visit.get('clinical_note', {}).get('plan', '정보 없음')}
            """
        else:
            integrated_doc += "방문 기록 없음\n"

        documents.append(Document(
            page_content=integrated_doc.strip(),
            metadata={
                "patient_id": patient['id'],
                "name": patient['name'],
                "gender": patient['gender'],
                "age": patient['age'],
                "department": department,
                "document_type": "integrated_record"
            }
        ))

        return documents

    def _process_document_batch(self, documents_batch):
        """
        문서 배치 처리 - 임베딩 생성 병렬화용
        """
        from langchain_community.vectorstores import FAISS

        # 문서를 청크로 분할
        chunks = self.text_splitter.split_documents(documents_batch)

        # 배치별 벡터 스토어 생성
        return FAISS.from_documents(chunks, self.embeddings)

    def create_vector_store(self, documents, store_name="medical_vector_store", batch_size=500):
        """
        벡터 스토어 생성 - 병렬 처리
        """
        if not documents:
            logger.warning("벡터 스토어를 생성할 문서가 없습니다.")
            return None

        logger.info(f"{len(documents)}개 문서로 벡터 스토어 생성 중...")

        # 문서를 청크로 분할
        chunks = self.text_splitter.split_documents(documents)
        logger.info(f"총 {len(chunks)}개의 청크 생성")

        from langchain_community.vectorstores import FAISS

        # 벡터 스토어 경로
        store_path = self.vector_store_path / store_name
        store_path.mkdir(parents=True, exist_ok=True)

        # 배치 처리
        if len(chunks) > batch_size:
            # 배치 생성
            batches = [chunks[i:i + batch_size] for i in range(0, len(chunks), batch_size)]
            logger.info(f"{len(batches)}개의 배치로 분할하여 처리")

            # 첫 번째 배치로 초기 벡터스토어 생성
            vectorstore = FAISS.from_documents(batches[0], self.embeddings)

            # 나머지 배치 처리
            for i, batch in enumerate(tqdm(batches[1:], desc="Processing batches")):
                batch_vs = FAISS.from_documents(batch, self.embeddings)
                vectorstore.merge_from(batch_vs)
                logger.info(f"배치 {i+2}/{len(batches)} 완료")
        else:
            # 배치가 필요 없는 경우 한 번에 처리
            vectorstore = FAISS.from_documents(chunks, self.embeddings)

        # 벡터 스토어 저장
        vectorstore.save_local(store_path)
        logger.info(f"벡터 스토어가 {store_path}에 저장되었습니다.")
        return vectorstore

    def load_vector_store(self, store_name="medical_vector_store"):
        """
        저장된 벡터 스토어 로드
        """
        from langchain_community.vectorstores import FAISS

        store_path = self.vector_store_path / store_name

        if not store_path.exists():
            logger.error(f"벡터 스토어 경로가 존재하지 않습니다: {store_path}")
            return None

        logger.info(f"{store_path}에서 벡터 스토어 로드 중...")

        try:
            vectorstore = FAISS.load_local(
                store_path,
                self.embeddings,
                allow_dangerous_deserialization=True
            )
            logger.info("벡터 스토어 로드 완료")
            return vectorstore
        except Exception as e:
            logger.error(f"벡터 스토어 로드 중 오류 발생: {e}")
            return None

    def search_similar_documents(self, query, vectorstore, k=5, filter_dict=None):
        """
        유사 문서 검색 (메타데이터 필터링 지원)
        """
        if not vectorstore:
            logger.error("유효한 벡터 스토어가 없습니다.")
            return []
        
        logger.info(f"쿼리로 검색 중: {query}")
        
        if filter_dict:
            # 메타데이터 필터 적용한 검색
            docs = vectorstore.similarity_search(
                query, 
                k=k,
                filter=filter_dict
            )
        else:
            # 기본 유사도 검색
            docs = vectorstore.similarity_search(query, k=k)
        
        return docs

    def create_vector_indices(self):
        """
        다양한 인덱스 및 벡터 스토어 구축
        """
        # 전체 의료 데이터 로드
        all_documents = self.load_medical_data()

        if not all_documents:
            logger.warning("벡터 인덱스를 생성할 문서가 없습니다.")
            return {}

        # 문서 타입별 필터링
        doc_types = {}
        doc_types["diagnosis"] = [doc for doc in all_documents if doc.metadata.get("document_type") == "diagnosis"]
        doc_types["medication"] = [doc for doc in all_documents if doc.metadata.get("document_type") == "medication"]
        doc_types["lab_results"] = [doc for doc in all_documents if doc.metadata.get("document_type") == "lab_result"]
        doc_types["visits"] = [doc for doc in all_documents if doc.metadata.get("document_type") == "visit"]

        # 진료과별 문서 필터링
        departments = set(doc.metadata.get("department", "") for doc in all_documents if doc.metadata.get("department"))
        dept_docs = {dept: [doc for doc in all_documents if doc.metadata.get("department") == dept] for dept in departments}

        # 병렬로 벡터 스토어 생성
        indices = {}

        # 메인 벡터 스토어 생성
        logger.info("메인 벡터 스토어 생성 중...")
        indices["general"] = self.create_vector_store(all_documents, "medical_vector_store")

        return indices

    def save_to_huggingface(self, vectorstore, username=None, token=None):
        """
        벡터 스토어를 Hugging Face Hub에 업로드

        Args:
            vectorstore: 업로드할 벡터 스토어 객체
            username: Hugging Face 사용자명 (선택 사항, 제공되지 않으면 토큰에서 추출)
            token: Hugging Face API 토큰

        Returns:
            성공 여부 (bool)
        """
        try:
            from huggingface_hub import HfApi, create_repo, upload_folder
            import tempfile
            import shutil
            import os

            # 저장소 ID 설정 (사용자명/MedicalVectorStore)
            if username:
                repo_id = f"{username}/MedicalVectorStore"
            else:
                # 토큰이 있으면 API를 통해 사용자명 가져오기
                if token:
                    api = HfApi(token=token)
                    username = api.whoami()["name"]
                    repo_id = f"{username}/MedicalVectorStore"
                else:
                    logger.error("사용자명이나 토큰이 제공되지 않았습니다.")
                    return False

            logger.info(f"벡터 스토어를 Hugging Face Hub의 {repo_id}에 업로드 중...")

            # 임시 디렉토리 생성
            with tempfile.TemporaryDirectory() as temp_dir:
                temp_dir_path = Path(temp_dir)

                # 벡터 스토어를 임시 디렉토리에 저장
                vectorstore.save_local(temp_dir_path)

                # Hugging Face 저장소 생성 (존재하지 않는 경우)
                try:
                    create_repo(repo_id=repo_id, token=token, private=False)
                    logger.info(f"저장소 {repo_id} 생성 완료")
                except Exception as e:
                    logger.info(f"저장소가 이미 존재하거나 생성 중 오류 발생: {e}")

                # 임시 디렉토리의 파일을 Hugging Face Hub에 업로드
                api = HfApi()
                api.upload_folder(
                    folder_path=temp_dir,
                    repo_id=repo_id,
                    repo_type="model",
                    token=token
                )

                logger.info(f"벡터 스토어가 성공적으로 Hugging Face Hub의 {repo_id}에 업로드되었습니다.")
                return True

        except Exception as e:
            logger.error(f"Hugging Face Hub 업로드 중 오류 발생: {e}")
            return False


# 메인 함수
def main():
    # 경로 설정
    current_dir = os.getcwd()  # 현재 폴더
    data_path = os.path.join(current_dir, "medical_data")  # 현재 폴더 내 medical_data
    vector_store_path = os.path.join(current_dir, "vector_stores")  # 현재 폴더 내 vector_stores
    model_cache_dir = "D:/huggingface_cache"  # D드라이브에 모델 캐시 저장

    # GPU 사용 가능 여부 확인
    use_gpu = torch.cuda.is_available()
    if use_gpu:
        logger.info(f"GPU 사용 가능: {torch.cuda.get_device_name(0)}")
    else:
        logger.info("GPU를 사용할 수 없습니다. CPU로 실행합니다.")

    # 벡터 스토어 객체 초기화
    vs_builder = MedicalVectorStore(
        data_path=data_path,
        vector_store_path=vector_store_path,
        model_cache_dir=model_cache_dir,
        use_gpu=use_gpu
    )

    # 의료 데이터 로드
    logger.info("의료 데이터 로드 중...")
    documents = vs_builder.load_medical_data()

    # 벡터 스토어 생성
    logger.info("벡터 스토어 구축 중...")
    vectorstore = vs_builder.create_vector_store(documents)

    if vectorstore:
        logger.info("벡터 스토어 구축 완료!")
        logger.info(f"벡터 스토어가 {vector_store_path}에 저장되었습니다.")

        # 간단한 검색 테스트
        test_query = "고혈압 환자의 최근 혈압 측정 기록"
        logger.info(f"테스트 쿼리 실행 중: {test_query}")
        
        results = vs_builder.search_similar_documents(test_query, vectorstore, k=3)
        logger.info(f"검색 결과: {len(results)}개 문서 찾음")
        
        # 결과 예시 출력
        for i, doc in enumerate(results):
            logger.info(f"결과 {i+1} - {doc.metadata.get('document_type')}: {doc.page_content[:150]}...")

        # Hugging Face Hub에 벡터 스토어 업로드 (선택 사항)
        upload_to_hf = input("Hugging Face Hub에 벡터 스토어를 업로드하시겠습니까? (y/n): ")
        
        if upload_to_hf.lower() == 'y':
            token = input("Hugging Face 토큰을 입력하세요: ")
            username = input("Hugging Face 사용자명을 입력하세요: ")
            
            if not token:
                logger.error("토큰이 제공되지 않았습니다.")
            elif not username:
                logger.error("사용자명이 제공되지 않았습니다.")
            else:
                # Hugging Face Hub에 업로드
                success = vs_builder.save_to_huggingface(vectorstore, username, token)
                
                if success:
                    repo_url = f"https://huggingface.co/{username}/MedicalVectorStore"
                    logger.info(f"벡터 스토어가 성공적으로 업로드되었습니다.")
                    logger.info(f"다음 URL에서 액세스할 수 있습니다: {repo_url}")
                else:
                    logger.error("벡터 스토어 업로드에 실패했습니다.")


if __name__ == "__main__":
    main()

2025-05-18 20:20:58,148 - INFO - GPU를 사용할 수 없습니다. CPU로 실행합니다.
2025-05-18 20:20:58,149 - INFO - Using device: cpu


2025-05-18 20:20:58,151 - INFO - Load pretrained SentenceTransformer: meta-llama/Llama-4-Maverick-17B-128E-Instruct
2025-05-18 20:20:58,380 - WARNING - No sentence-transformers model found with name meta-llama/Llama-4-Maverick-17B-128E-Instruct. Creating a new one with mean pooling.
Fetching 55 files:   0%|          | 0/55 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
2025-05-18 20:21:01,105 - WARNING - Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the pac